In [36]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [3]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
#set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [23]:
def get_question(qid):
    for i, item in devdf.iterrows():
        for q in item['questions']:        
            if q['uid'] == qid:
                table = item['table']['table']
                return (item['table'], q)
    return (None, None)

In [42]:
def gen_code3(llm, messages, question, value_list): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"value_list": values, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return ("", code)

In [43]:
 messages=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a list of annotated values. You will receive the financial report as an annotated value list and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as a list of annotated values: {value_list}"
        ),         
        (
          "ai",
          "Ok, I received the value list."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
          "Generate a Python function 'run(value_list)' that can answer the question using the list of annotated values! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            "If the question is about calculating the average value in a given year, you must calculate the average between the given year and the previous one. It is aka Average Book Value. A value always represents year-end value.  "+
            "If the question is about calculating the change in average values in years, you must calculate the change in year average values before difference calculation. "
           
            "Do not generate explanation, nor example code, just the function."
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ]     


In [47]:

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0, top_p = 1.0, seed=11)

devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

#tid = 'ba6783f3-8207-419a-b407-3f688682caef'
#qid = 'ba6783f3-8207-419a-b407-3f688682caef'
#qid = 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'

#qid = 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'
#qid = 'bf7abd62-d9cd-48d2-8826-1457684019a3'
#qid = '4d259081-6da6-44bd-8830-e4de0031744c'
#qid = 'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20'

#qid = '7cd3aedf-1291-4fea-bc9d-a25c65727b7b'
qid = '22e20f25-669a-46b9-8779-2768ba391955'
#qid = '65ec782c-691e-45df-b541-caecb85154ff'
#qid = 'a983501d-2eec-486d-9661-e520c7c8af5e'


_table, _q =  get_question(qid)
table = _table['table']

[print(r) for r in table]

q = _q['question']
print(q)
values = table_convert.convert_table(table)

code = gen_code3(llm, table, q, messages )
print(code)

(v, s) = exec_code(code, values)
print(_q['derivation'])
if v == _q['answer']:
    print("Success: " + str(v))
else:    
    print("failure: " + str(v), 'good answer: ', _q['answer'])    

['', '2019', '2018', '2017']
['', '€m', '€m', '€m']
['Cash generated by operations (refer to note 18)', '14,182', '13,860', '13,781']
['Capital additions', '(7,227)', '(7,321)', '(7,675)']
['Working capital movement in respect of capital additions', '(89)', '171', '(822)']
['Disposal of property, plant and equipment', '45', '41', '43']
['Restructuring payments', '195', '250', '266']
['Other', '(35)', '–', '34']
['Operating free cash flow', '7,071', '7,001', '5,627']
['Taxation', '(1,040)', '(1,010)', '(761)']
['Dividends received from associates and investments', '498', '489', '433']
['Dividends paid to non-controlling shareholders in subsidiaries', '(584)', '(310)', '(413)']
['Interest received and paid', '(502)', '(753)', '(830)']
['Free cash flow (pre-spectrum)', '5,443', '5,417', '4,056']
['Licence and spectrum payments', '(837)', '(1,123)', '(474)']
['Restructuring payments', '(195)', '(250)', '(266)']
['Free cash flow', '4,411', '4,044', '3,316']
[(4,411+4,044)/2] -[(4,044+3,316)

In [ ]:
qids = [
#'ba6783f3-8207-419a-b407-3f688682caef',
'a0414f81-8dc2-44b2-a441-2c9d9c805c4d',
'bf7abd62-d9cd-48d2-8826-1457684019a3',
'4d259081-6da6-44bd-8830-e4de0031744c',
'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20',

'7cd3aedf-1291-4fea-bc9d-a25c65727b7b',
'22e20f25-669a-46b9-8779-2768ba391955',
#'65ec782c-691e-45df-b541-caecb85154ff',
#'a983501d-2eec-486d-9661-e520c7c8af5e'
]
def get_answer(table, question):
    values = table_convert.convert_table(table)
    p, code = gen_code2(q, values)
    (v, s) = exec_code(code, values)
    return (v, s)
    
for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    q = _q['question']
    print(q, end='')
    r = []
    for i in [0,1,2]:        
       r.append(get_answer(table, q))
    
    if r[0][0] == r[1][0]:
        v = r[0][0]
    if r[1][0] == r[2][0]:
        v = r[1][0]
    if r[0][0] == r[2][0]:
        v = r[0][0] 
    
    #print(_q['derivation'])
    if v == _q['answer']:
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )  